In [14]:
from seleniumbase import Driver
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd

Selecting Url

In [15]:
main_url = "https://www.sahibinden.com"
base_url = f"{main_url}/fiat"

opening browser

In [16]:
# Set up Browser
ua = UserAgent()
user_agent = ua.random
browser = user_agent.split()
isphone = False
if "iphone" in browser or "android" in browser:
    isphone = True
driver = Driver(uc=True , agent = user_agent , headed=True)

# Going to page
if isphone : 
    driver.get(main_url)
    driver.get(driver.current_url + "/fiat")
else :
    driver.get(base_url)

waiting for cloudflare to initialize

In [17]:
sleep(1)

creating needed variables

In [18]:
df = pd.DataFrame()
gone_in = False
results = {}

starting data scraping

In [19]:
while True:
    # Finding all the cars by looking for all "searchResultsItem" css elements
    # cars = []
    if isphone :
        cars = driver.find_elements(By.CSS_SELECTOR, '[class*="search-result-item"]')
    else :
        cars = driver.find_elements(By.CSS_SELECTOR, '[class*="searchResultsItem"]')
    # For each car, get the available details and add them to the dataframe
    for car in cars:
        try:
        # if car.text != "":
            # Open the car details in a new tab or window
            # Because sahibenden restes the page after going back so the "cars" get's lost
            driver.execute_script("window.open(arguments[0]);", car.find_element(By.TAG_NAME, 'a').get_attribute('href'))
            sleep(1)

            # Switch to the new tab or window
            driver.switch_to.window(driver.window_handles[-1])
            sleep(1)
            # Safety element for exceptions
            gone_in = True
            sleep(1)

            # Wait for the car_details element to be present
            if isphone :
                car_details = driver.find_element(By.CSS_SELECTOR, '[class*="classified-info-list"]').text.split('\n')
                sleep(1)
                location = driver.find_element(By.CSS_SELECTOR, '[class*="location-info"]')
            else :
                car_details = driver.find_element(By.CSS_SELECTOR, '[class*="classifiedInfo"]').text.split('\n')
            
            sleep(1)
            # Fill car data
            if isphone :
                fullAdrs = location.text.split(" / ")
            else :
                fullAdrs = car_details[2].split(" / ")
                price = car_details[0]
            # Assign each part to a different variable
            city, district, neighborhood = fullAdrs
            
            results = {}
            # Assign only available data  
            if isphone :
                results['price'] = [price]
            results['city'] = [city]
            results['district'] = [district]
            results['neighborhood'] = [neighborhood]
            for i in range(0, len(car_details) - 1, 2):
                key = car_details[i].strip()  # Remove any leading/trailing spaces from the key
                if i + 1 < len(car_details):  # Make sure there is a value after the key
                    value = car_details[i + 1].strip()  # Remove any leading/trailing spaces from the value
                    results[key] = [value]
            

            # Append the new DataFrame to the existing DataFrame
            sleep(1)
            df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)

            sleep(1)
            # Close the current tab or window
            driver.close()

            # Switch back to the original tab or window
            driver.switch_to.window(driver.window_handles[0])
            sleep(1)
            gone_in = False

        except Exception as err:
            # Switch back to the original tab or window
            # Check if new window is open and close it
            print(f"page error : {err}")
            if gone_in:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                gone_in = False

            sleep(1)
            print("Exception")
            continue
            # break
    try:
        # Find the next page button and because on the last tab it disappears it gives an exeption and goes out 
        if isphone :
            next_page_button = driver.find_element(By.CSS_SELECTOR, '[class*="next-page"]')
        else :
            next_page_button =  driver.find_element(By.LINK_TEXT, 'Sonraki')
        # driver.click('a[title="Sonraki"]')
        next_page_button.click()
        sleep(2)
    except  Exception as err:
        print("Exception on next page" + err)
        break


TypeError: can only concatenate str (not "NoSuchElementException") to str

Save the data to CSV file

In [ ]:
df

In [8]:
df.to_csv('all_car_listings.csv', index=False)